## Step 1: Install dependencies

In [1]:
!pip install importlib-metadata==4.12.0 # To overcome an issue with importlib-metadata https://stackoverflow.com/questions/73929564/entrypoints-object-has-no-attribute-get-digital-ocean
!pip install gym[box2d]
!pip install stable-baselines3[extra]
!pip install pyglet==1.5.1
!pip install ale-py==0.7.4 # To overcome an issue with gym (https://github.com/DLR-RM/stable-baselines3/issues/875)
!pip install botorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached importlib_metadata-4.12.0-py3-none-any.whl (21 kB)
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.13.0
    Uninstalling importlib-metadata-4.13.0:
      Successfully uninstalled importlib-metadata-4.13.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
stable-baselines3 1.7.0 requires importlib-metadata~=4.13, but you have importlib-metadata 4.12.0 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached importlib_metadata-4.13.0-py3-none-any.whl (23 kB)
  Attempting uninstall: importlib-metadata
    Found existing installati

## Step 2: Import libraries

In [2]:
import os
import torch
import numpy as np
import plotly
import plotly.graph_objects as go

import gym

from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_vec_env

## Step 3: Define hyperparameters

In [3]:
rl_env_name = 'CartPole-v1'

In [4]:
# Define initial values for PPO hyperparameters
policy = 'MlpPolicy'
env = make_vec_env(rl_env_name)
learning_rate = 0.05
n_steps = 1024
batch_size = 64
n_epochs = 4
gamma = 0.999
gae_lambda = 0.98
ent_coef = 0.01

# Define here the list of parameters to tune
hyperparams_list = [learning_rate]
# Define the lower bounds of the parameters
lower_bounds = [0.0001]
# Define the upper bounds of the parameters
upper_bounds = [0.3]

Convert lists to tensors

In [5]:
# Create tensors with the hyperparameters configurations and bounds for BOTorch to use
hyperparams_tensor = torch.DoubleTensor([hyperparams_list])
bounds_tensor = torch.DoubleTensor([lower_bounds, upper_bounds])

## Step 4: Create initial results

In [6]:
def get_hyp_values(hyperparams_tensor):
  '''
  Returns a tuple of values from a tensor containing a hyperparameter configuration

          Parameters:
                  hyperparams_tensor (torch.DoubleTensor): A tensor of size 1xn (1 row, n columns) with n being the number of hyperparameters to tune
          
          Returns:
                  hyperparams_tuple (tuple): A tuple with the unpacked values of the hyperparams_tensor 

  '''
  hyperparams_list = [hyperparams_tensor[0][i].item() for i in range(len(hyperparams_tensor[0]))]
  hyperparams_tuple = tuple(hyperparams_list)
  return tuple(hyperparams_list)


def create_model(hyperparams,
                 policy='MlpPolicy',
                 env='CartPole-v1'):
  '''
  Returns a PPO model given a policy, environment, and hyperparameters of PPO

          Parameters:
                  hyperparams (torch.DoubleTensor): A tensor of size 1xn (1 row, n columns) with n being the number of hyperparameters to tune
                  policy (str): The NN to train with PPO in the environment. Default is 'MlpPolicy'
                  env (stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv): Specifies the gym environment to use for the training

          Returns:
                  model (stable_baselines3.ppo.ppo.PPO): The model to train
  '''
  lr,  = get_hyp_values(hyperparams)
  model = PPO(policy = policy,
              env = env,
              learning_rate = lr,
              n_steps = 1024,
              batch_size = 64,
              n_epochs = 4,
              gamma = 0.999,
              gae_lambda = 0.98,
              ent_coef = 0.01,
              verbose=0)
  
  return  model

def train_model(model, timesteps=10000):
  '''
  Trains a PPO model during a number of timesteps
          
          Parameters:
                  model (stable_baselines3.ppo.ppo.PPO): The model to train
                  timesteps (int): The number of timesteps used to train the model

          Returns:
                  None
  '''
  model.learn(total_timesteps=timesteps)
  return

def evaluate_model(model, rl_env_name='CartPole-v1', n_eval_episodes=10):
  '''
  Evaluates the model for a number of episodes in a specified environment, this environment MUST be the same as the one the model has been trained in.

          Parameters:
                  model (stable_baselines3.ppo.ppo.PPO): The model to train
                  rl_env_name (str): The name of the gym environment where the model has been trained
                  n_eval_episodes (int): The number of episodes for which the model will be evaluated to obtain a mean and standard deviation

          Returns:
                  mean_reward (torch.DoubleTensor): A tensor of size 1x1 (1 row, 1 column) containing the mean_reward
  '''
  eval_env = gym.make(rl_env_name)
  mean_reward, std_reward = evaluate_policy(model, 
                                            eval_env, 
                                            n_eval_episodes=n_eval_episodes, 
                                            deterministic=True)
  
  print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")
  return torch.DoubleTensor([[mean_reward]])


In [7]:
# Create the model to train
model = create_model(hyperparams_tensor,
                     policy, 
                     env)

Train the model for the first time to have our initial evaluation of the mean reward function that we will use to measure the performance of a model.

In [8]:
# Train for timesteps
train_timesteps = 100000
train_model(model, timesteps=train_timesteps)

Evaluate the model, create the rewards tensor (init_y) and get the best reward

In [9]:
# Evaluate the model to get our first value of the mean reward function
rewards_tensor = evaluate_model(model, rl_env_name)
# Set the initial reward as the best reward
best_reward = rewards_tensor.max().item()

mean_reward=9.10 +/- 0.5385164807134504


/usr/local/lib/python3.8/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


## Step 4: Use Gaussian Process with initial data

We set which model and which likelihood will we use. In our case we will use a classic Gaussian process and compute its hyper-parameters using the exact marginal log likelihood (which can produce overfitting when points are reduced but well...)

In [10]:
from botorch.models import SingleTaskGP, ModelListGP
from gpytorch.mlls import LeaveOneOutPseudoLikelihood, ExactMarginalLogLikelihood
from botorch.models.transforms.outcome import Standardize
from botorch import fit_gpytorch_model
single_model = SingleTaskGP(hyperparams_tensor,
                            rewards_tensor)
mll = ExactMarginalLogLikelihood(single_model.likelihood, 
                                 single_model)

Now that our model is declared, we fit the previous points with the Gaussian process setting its hyperparameters via Exact Marginal log likelihood of the points. The output shows the default covariance function used by the GP and its hyper-hyperparameters. It also shows the Gaussian likelihood used and the homoskedastic noise added to the Matern Kernel to capture the noise of the data. 

In [11]:
fit_gpytorch_model(mll)

ExactMarginalLogLikelihood(
  (likelihood): GaussianLikelihood(
    (noise_covar): HomoskedasticNoise(
      (noise_prior): GammaPrior()
      (raw_noise_constraint): GreaterThan(1.000E-04)
    )
  )
  (model): SingleTaskGP(
    (likelihood): GaussianLikelihood(
      (noise_covar): HomoskedasticNoise(
        (noise_prior): GammaPrior()
        (raw_noise_constraint): GreaterThan(1.000E-04)
      )
    )
    (mean_module): ConstantMean()
    (covar_module): ScaleKernel(
      (base_kernel): MaternKernel(
        (lengthscale_prior): GammaPrior()
        (raw_lengthscale_constraint): Positive()
      )
      (outputscale_prior): GammaPrior()
      (raw_outputscale_constraint): Positive()
    )
  )
)

Now we declare the acquisition function that is going to be computed using the predictive distribution of the previous Gaussian process in all the input space. We will use the upper confidence bound.

In [12]:
from botorch.acquisition.monte_carlo import qExpectedImprovement #use the noisy version if the problem has noise

EI = qExpectedImprovement(model=single_model,
                          best_f=best_reward)

We will now optimize the acquisition function, all the hyper parameters here are a good heuristic default to try and find the global optima of the acquisition function

In [13]:
from botorch.optim import optimize_acqf

candidates, _ = optimize_acqf(acq_function=EI, 
                              bounds=bounds_tensor, 
                              q=1, 
                              num_restarts=200,
                              raw_samples=512, 
                              options={"batch_limit": 5, "maxiter": 200})
candidates

tensor([[0.3000]], dtype=torch.float64)

We define functions to visualize GP

In [14]:
def compute_predictive_distribution(single_model, 
                                    best_init_y, 
                                    l_bound=-2.,
                                    h_bound=10.,
                                    resolution=1000):
  '''
  Computes the predictive mean and variance

          Parameters:
                  single_model (botorch.models.gp_regression): A Gaussian Process regression model
                  best_init_y (float): The best value of the objective function obtained
                  l_bound (float): The lower bound of the hyperparameter
                  h_bound (float): The upper bound of the hyperparameter
                  resolution (int): The number of points used to discretize the bounded space
          Returns:
                  result_tensor (torch.Tensor): A tensor of size 'resolution' (a simple 1d array with 1 value) containing the predictive mean of the objective function in each discretized point
                  variances_tensor (torch.Tensor): A tensor of size 'resolution' (a simple 1d array with 1 value) containig the predictive variances of the objective function in each discretized point
  '''
  linspace = torch.linspace(l_bound, h_bound, steps=resolution)
  x_test = torch.tensor([linspace[0]]).unsqueeze(-1)
  result = []
  variances = []
  for x in linspace:
    x_test = torch.tensor([x]).unsqueeze(-1)
    result.append(single_model.posterior(x_test).mean)
    variances.append(single_model.posterior(x_test).variance)
  
  result_tensor = torch.tensor(result)
  variances_tensor = torch.tensor(variances)
  return result_tensor, variances_tensor


def print_predictive_mean(predictive_mean, 
                          predictive_variance,
                          iteration, 
                          l_bound=-2.,
                          h_bound=10.,
                          resolution=1000,
                          suggested=None,
                          old_obs=[], 
                          old_values=[]):
  '''
  Plots the predictive mean and variance of the objective function.

          Parameters:
                  result_tensor (torch.Tensor): A tensor of size 'resolution' (a simple 1d array with 1 value) containing the predictive mean of the objective function in each discretized point
                  variances_tensor (torch.Tensor): A tensor of size 'resolution' (a simple 1d array with 1 value) containig the predictive variances of the objective function in each discretized point
                  iteration (int): The iteration number of the bayesian optimization loop
                  l_bound (float): The lower bound of the hyperparameter to optimize
                  h_bound (float): The upper bound of the hyperparameter to optimize
                  resolution (int): The number of points used to discretize the bounded space
                  suggested (torch.DoubleTensor): A tensor of size 1x1 containing the next point to evaluate
                  old_obs (list): A float list with the previous points evaluated
                  old_values (list): A float list with the previous evaluations of the objective function
          
          Returns:
                  None
  '''
  
  x = torch.linspace(l_bound, h_bound, steps=resolution).detach().numpy()
  x_new = x.reshape((resolution,-1))
  z = predictive_mean
  max_predictive_mean = x[((predictive_mean == predictive_mean.max().item()).nonzero(as_tuple=True)[0])]

  fig = go.Figure()

  fig.add_trace(go.Scatter(x=x, y= predictive_mean + np.sqrt(predictive_variance),
                                     mode='lines',
                                     line=dict(color="#19D3F3",width =0.1),
                                     name='upper bound'))
  fig.add_trace(go.Scatter(x=x, y= predictive_mean,
                         mode='lines',
                         line=dict(color="blue"),
                         fill='tonexty',
                         name='predictive mean'))
  fig.add_trace(go.Scatter(x=x, y= predictive_mean - np.sqrt(predictive_variance),
                         mode='lines',
                         line=dict(color="blue", width =0.1),
                         fill='tonexty',
                         name='lower bound'))
  
  
  
  fig.update_layout(title="GP Predictive distribution. Iteration " + str(iteration), xaxis_title="learning_rate", yaxis_title="mean_reward", showlegend=False)

  if(suggested==None):
    fig.add_vline(x=max_predictive_mean, line_width=3, line_color="red")
  else:
    fig.add_vline(x=float(suggested[0][0]), line_width=3, line_color="red")  

  if(len(old_obs)>0):
    fig.add_trace(go.Scatter(x=old_obs, y=old_values, mode = 'markers', marker_color="black", marker_size=10))

  fig.show()

In [15]:
# Test our predictive distribution function
predictive_mean, predictive_variance = compute_predictive_distribution(single_model, 
                                                                       best_reward,
                                                                       l_bound = lower_bounds[0],
                                                                       h_bound = upper_bounds[0])
# Plot the predictive distribution function
print_predictive_mean(predictive_mean, 
                      predictive_variance, 
                      1, 
                      l_bound = lower_bounds[0],
                      h_bound = upper_bounds[0])

We now have all the code of an iteration so we just put it in a loop. To do so: We just wrap previous code into a function.



In [16]:
def get_next_hyperparameters(hyperparams_tensor,
                             rewards_tensor,
                             best_reward,
                             bounds_tensor,
                             n_points=1):
  single_model = SingleTaskGP(hyperparams_tensor,
                              rewards_tensor)
  mll = ExactMarginalLogLikelihood(single_model.likelihood, 
                                   single_model)
  fit_gpytorch_model(mll)

  EI = qExpectedImprovement(model=single_model,
                            best_f=best_reward)
  
  candidates, _ = optimize_acqf(acq_function=EI,
                                bounds=bounds_tensor,
                                q=n_points, 
                                num_restarts=100,
                                raw_samples=512,
                                options={"batch_limit": 5, "maxiter": 200})
  
  return candidates

Finally, we embed the previous code into the Bayesian optimization loop

In [18]:
n_iterations = 5

n_candidates_per_iteration = 1
candidates = []
results = []
best_candidate = learning_rate
for i in range(n_iterations):
  print(f"Number of iterations done: {i}")

  # Create our GP regression model
  single_model = SingleTaskGP(hyperparams_tensor,
                              rewards_tensor)
  
  # Define the function to obtain the marginal probabilities
  mll = ExactMarginalLogLikelihood(single_model.likelihood, 
                                   single_model)
  
  # Fit our GP Regression model
  fit_gpytorch_model(mll)

  # Define our acquisition function
  EI = qExpectedImprovement(model=single_model,
                            best_f=best_reward)
  
  # Get our next candidates
  hyp_candidates, _ = optimize_acqf(acq_function=EI,
                                    bounds=bounds_tensor,
                                    q=n_candidates_per_iteration, 
                                    num_restarts=100,
                                    raw_samples=512,
                                    options={"batch_limit": 5, "maxiter": 200})
  
  # Create a PPO RL model to evaluate with the new hyperparameters candidate
  model = create_model(hyp_candidates,
                       policy,
                       env)
  
  # Train our PPO RL model
  train_model(model, train_timesteps)
  new_reward = evaluate_model(model, rl_env_name)
  
  # Compute the predictive mean of the GP Regression model
  predictive_mean, predictive_variance = compute_predictive_distribution(single_model, 
                                                                         best_reward,
                                                                         l_bound = lower_bounds[0],
                                                                         h_bound = upper_bounds[0])
  
  # Plot our GP Regression model
  print_predictive_mean(predictive_mean, 
                        predictive_variance, 
                        i, 
                        l_bound = lower_bounds[0],
                        h_bound = upper_bounds[0],
                        suggested = hyp_candidates,
                        old_obs = candidates,
                        old_values = results )
  
  print(f"New candidates are: {hyp_candidates}")

  # Update our hyperparameters history adding a new row to the hyperparameters tensor containing the configuration selected in this iteration
  hyperparams_tensor = torch.cat([hyperparams_tensor, hyp_candidates])
  # Update our rewards history adding a new row to the rewards tensor containing the configuration selected in this iteration
  rewards_tensor = torch.cat([rewards_tensor, new_reward])

  # Update our best reward and hyperparameters if necessary
  if rewards_tensor.max().item() > best_reward:
    best_reward = rewards_tensor.max().item()
    best_candidate = hyp_candidates

  print(f"Best learning rate is {best_candidate} and gets this mean reward: {best_reward}")
  # Update our lists used to plot
  candidates.append(float(hyp_candidates[0][0]))
  results.append(float(new_reward[0][0]))

Number of iterations done: 0


/usr/local/lib/python3.8/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning:

Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.



mean_reward=9.50 +/- 0.6708203932499369


New candidates are: tensor([[0.0317]], dtype=torch.float64)
Best learning rate is 0.05 and gets this mean reward: 86.2
Number of iterations done: 1


/usr/local/lib/python3.8/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning:

Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.



...

mean_reward=9.20 +/- 0.8717797887081347


New candidates are: tensor([[0.1917]], dtype=torch.float64)
Best learning rate is tensor([[0.0065]], dtype=torch.float64) and gets this mean reward: 500.0
Number of iterations done: 4


/usr/local/lib/python3.8/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning:

Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.



mean_reward=9.70 +/- 0.6403124237432849


New candidates are: tensor([[0.1606]], dtype=torch.float64)
Best learning rate is tensor([[0.0065]], dtype=torch.float64) and gets this mean reward: 500.0
